###  Import

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date
from time import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
sns.set()


In [2]:
# Import dataset
df = pd.read_csv('PRSA_data_2010.1.1-2014.12.31.csv')

In [3]:
df.info()
# we have ~44k samples (~4.3 MB)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43824 entries, 0 to 43823
Data columns (total 13 columns):
No       43824 non-null int64
year     43824 non-null int64
month    43824 non-null int64
day      43824 non-null int64
hour     43824 non-null int64
pm2.5    41757 non-null float64
DEWP     43824 non-null int64
TEMP     43824 non-null float64
PRES     43824 non-null float64
cbwd     43824 non-null object
Iws      43824 non-null float64
Is       43824 non-null int64
Ir       43824 non-null int64
dtypes: float64(4), int64(8), object(1)
memory usage: 4.3+ MB


In [4]:
df.head()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0


### Wrangle

In [5]:
# Drop `No` column (it's just an index)
df.drop('No', axis=1, inplace=True)

In [6]:
# Rename columns
df.columns = ['year', 'month', 'day', 'hour', 'pm25', 'dewp', 'temp', 'pres', 'wind_dir', 'wind_speed', 'snow_hours', 'rain_hours']
df.head()

,year,month,day,hour,pm25,dewp,temp,pres,wind_dir,wind_speed,snow_hours,rain_hours
0,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0


In [7]:
# Set DateTime Index
df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
df.set_index('date', inplace=True)
df.head()
df.tail()

,year,month,day,hour,pm25,dewp,temp,pres,wind_dir,wind_speed,snow_hours,rain_hours
date,,,,,,,,,,,,
2010-01-01 00:00:00,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
2010-01-01 01:00:00,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2010-01-01 02:00:00,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
2010-01-01 03:00:00,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
2010-01-01 04:00:00,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0


,year,month,day,hour,pm25,dewp,temp,pres,wind_dir,wind_speed,snow_hours,rain_hours
date,,,,,,,,,,,,
2014-12-31 19:00:00,2014,12,31,19,8.0,-23,-2.0,1034.0,NW,231.97,0,0
2014-12-31 20:00:00,2014,12,31,20,10.0,-22,-3.0,1034.0,NW,237.78,0,0
2014-12-31 21:00:00,2014,12,31,21,10.0,-22,-3.0,1034.0,NW,242.70,0,0
2014-12-31 22:00:00,2014,12,31,22,8.0,-22,-4.0,1034.0,NW,246.72,0,0
2014-12-31 23:00:00,2014,12,31,23,12.0,-21,-3.0,1034.0,NW,249.85,0,0


In [8]:
# Drop redundant information (year, month, day, and hour) that is now in the datetime index
df = df[['pm25', 'dewp', 'temp', 'pres', 'wind_speed', 'snow_hours', 'rain_hours']]

In [9]:
# Check for missing values
print('Missing values (%)')
print('------------------')
df.isnull().sum()/df.shape[0]*100

Missing values (%)
------------------


pm25          4.716594
dewp          0.000000
temp          0.000000
pres          0.000000
wind_speed    0.000000
snow_hours    0.000000
rain_hours    0.000000
dtype: float64

Less than 5% of data is missing. Let's just drop those points.


In [10]:
# Drop missing values
df.dropna(how='any', inplace=True)

df.shape
df.head()

(41757, 7)

,pm25,dewp,temp,pres,wind_speed,snow_hours,rain_hours
date,,,,,,,
2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,1.79,0,0
2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,2.68,0,0
2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,3.57,0,0
2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,5.36,1,0
2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,6.25,2,0


This brought us down to ~42k obs.

In [11]:
# Save cleaned dataset for faster accessibility in the future
filename = 'persistence/cleaned_data_' + str(date.today()) + '.csv'
df.to_csv(filename, date_format='%Y-%m-%d %H:%M:%S')